## 3.Data Preparation

On the data preparation stage the objective is prepare the data for the modeling stage. To accoplish this objective, we will utilise the tasks raised by the preceding stage, the data analysis stage. At the end of the third stage we will have the data tables to proceed with the modeling. 

### A)Selecting the data

Let's start loading the raw data given by the Kaggle competition website.

In [1]:
import pandas as pd
#loading the data
gender_submission = pd.read_csv("../data/gender_submission.csv")
test_ds = pd.read_csv("../data/test.csv")
train_ds = pd.read_csv("../data/train.csv")

In [2]:
train_ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


the analysis pointed useless columns: PassagerId, Name. Cabin and ticket, maybe is not useless but has missing values and alphabetic caracters and will be droped to.

In [3]:
columns_droped = ["PassengerId", "Ticket", "Cabin", "Name"]
train_ds.drop(labels=columns_droped, axis=1, inplace=True)
train_ds.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Making the same changes on the test dataset.

In [4]:
test_ds.drop(labels=columns_droped, axis=1, inplace=True)
test_ds.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


### B)Missing Values

The cabin, age, embarked are the three columns with missing values. The column cabin is already removed by the last procedure. Next we opted for changing the missing value with the mean of the column. This is a simple stategy to handle the missing values. The reason behind selecting this strategy is two: there is no easy way to determine the age with the others columns, and the data available is to small to drop the missing. There is more than 10% of available data with missing age, droping the rows is not a valid option. 

In [5]:
train_ds['Age'].mean()

29.69911764705882

Let's put change the missing values to 30.

In [6]:
train_ds['Age'].fillna(value=30, inplace=True)
test_ds['Age'].fillna(value=30, inplace=True)

There is two values missing in the embarked column for the training set, we will just drop them.

In [7]:
train_ds.drop(train_ds[train_ds['Embarked'].isnull()].index, inplace=True)

The test doesn't has any missing value for the embarked column.

### C)Transform data

In [8]:
train_ds['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

Some columns has categorical values which needs to be transformed. We will use one hot enconding.

In [18]:
from sklearn.preprocessing import OneHotEncoder
cat_columns = ['Pclass', 'Sex', 'Embarked']
one = OneHotEncoder(sparse_output=False).fit(X=train_ds[cat_columns])

In [19]:
one.get_feature_names_out()

array(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'], dtype=object)

In [20]:
#Transforming and adding the new columns on the data set
train_ds[one.get_feature_names_out()] = one.transform(train_ds[cat_columns])
test_ds[one.get_feature_names_out()] = one.transform(test_ds[cat_columns])

Verifying the transformation.

In [22]:
train_ds.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,male,22.0,1,0,7.2500,S,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1,1,female,38.0,1,0,71.2833,C,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1,1,female,35.0,1,0,53.1000,S,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,3,male,35.0,0,0,8.0500,S,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


Now we can drop the original columns.

In [31]:
train_ds.drop(one.feature_names_in_, axis=1, inplace=True)
test_ds.drop(one.feature_names_in_, axis=1, inplace=True)

In [32]:
train_ds.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,26.0,0,0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


The last transformation is apply some scaler on the numeric features. On the scenario presented by the data analysis a simple MinMaxScaler will work good. We can see that without the standardization, the Fare probaly will have a greater influence on the model because of the bigger numbers. Applying the standardization, we eliminate the influence by bigger numbers. We can see this as a 'fare play' between the features, every feature is treated equal by the model.

In [33]:
from sklearn.preprocessing import MinMaxScaler

#selecting the numeric features.
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']

mm_scaler = MinMaxScaler().fit(X=train_ds[numeric_features])

In [34]:
train_ds[numeric_features] = mm_scaler.transform(train_ds[numeric_features])
test_ds[numeric_features] = mm_scaler.transform(test_ds[numeric_features])

In [36]:
train_ds.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0.271174,0.125,0.0,0.014151,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1,0.472229,0.125,0.0,0.139136,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,0.321438,0.000,0.0,0.015469,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1,0.434531,0.125,0.0,0.103644,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,0.434531,0.000,0.0,0.015713,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


Before ending this section, we will save the scaler and the encoder.

In [37]:
import pickle

with open("../var/hotencoder", "wb") as f:
    pickle.dump(one, f)
with open("../var/scaler", "wb") as f:
    pickle.dump(mm_scaler,f)

### D)Spliting the data

Now we will split the data in y(targets) and X(features), respectively for the train and test data set.

In [44]:
y_train = train_ds['Survived']
x_train = train_ds.drop('Survived', axis=1)

#Test doesn't have the target in Kaggle competition
x_test = test_ds

Saving the data.

In [45]:
data_path = "../data"
y_train.to_csv(f"{data_path}/y_train.csv")
x_train.to_csv(f"{data_path}/x_train.csv")
x_test.to_csv(f"{data_path}/x_test.csv")